In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import glob
import os
import scipy.stats
from matplotlib import cm
import seaborn as sns

In [2]:
def get_pressure_weighted(x):
    dPref = (x.plev.values[0]-x.plev.values[-1])  #(p-ps)
    integral = []
    for i in range(len(x.plev)): #Integral of variable from P to Ps calculated as area between each pressure variable trapezoid then summed
        if i+1 < len(x.plev):
            area=((x.loc[dict(plev=x.plev.values[i])] + x.loc[dict(plev=x.plev.values[i+1])])/2)*(x.plev.values[i]-x.plev.values[i+1])
            integral.append(area)
    pw = (sum(integral))/dPref
    return(pw)

In [3]:
def low_pass_weights(window, cutoff):
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

wgts = low_pass_weights(41, 1/10)
weight = xr.DataArray(list(wgts), dims=['window'])

In [10]:
def jettracker(x):
    x = x.fillna(0)
    x = x.rolling(time=41, center=True).construct('window').dot(weight)
    x = x.dropna(dim='time',how='all')
    limit = np.quantile(x.values,0.9)
    x = x.where((x>=limit))
    x = x/x
    x = x.fillna(0)
    x = x.resample(time='QS-DEC').mean(dim='time',skipna=True)
    return x


In [20]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        ua = glob.glob(str(model)+"/r1i1p1/day/2deg/ua_*")
        va = glob.glob(str(model)+"/r1i1p1/day/2deg/va_*")
        try:
            test = va[0]
            avail[model.split('/')[-1]] = [ua,va]
        except:
             pass
    return avail

In [35]:
files = get_files()
files['NOAA'] = ['/home/pmarsh/NOAA_2deg/NOAA/NOAA_ua_850_700_2deg.nc','/home/pmarsh/NOAA_2deg/NOAA/NOAA_va_850_700_2deg.nc']

In [40]:
files['ERA5'] = [glob.glob("/home/pmarsh/NOAA_2deg/ERA5/ERA5_ua_850_700_2deg.nc"),glob.glob("/home/pmarsh/NOAA_2deg/ERA5/ERA5_va_850_700_2deg.nc")]

In [24]:
jetdic={}

In [41]:
for model in files:
    print(model)
    U = xr.open_mfdataset(files[model][0])
    V = xr.open_mfdataset(files[model][1])
    levels=[85000,70000]
    if model == 'NOAA':
        U = U.rename({'uwnd':'ua'})
        U = U.rename({'level':'plev'})
        V = V.rename({'vwnd':'va'})
        V = V.rename({'level':'plev'})
        levels=[850,700]
    elif model == 'ERA5':
        U = U.rename({'level':'plev'})
        V = V.rename({'level':'plev'})
        levels=[850,700]
    x = np.sqrt(np.square(U.ua) + np.square(V.va))
    x['ws'] = x
    x = x.sel(plev=levels)
    x = x.sel(lat = slice(-75,-15))
    x = xr.concat([x.sel(lon = slice(0,30)),x.sel(lon = slice(320,360))],dim='lon')
    x.coords['lon'] = (x.coords['lon'] + 180) % 360 - 180
    x = get_pressure_weighted(x)
    if model == 'ERA5':
        x = x.sel(time=slice('1980', '2020'))
    else:
        x = x.sel(time=slice('1950', '2005'))
    jetdic[model] = jettracker(x)

ERA5


In [ ]:
results=[]
for index in jetdic:
    reference = index
    for index in jetdic:
        MAE=[]
        for season in ['DJF','MAM','JJA','SON']:
            ref = jetdic[reference].where(jetdic[reference].time.dt.season==season).mean(dim='time')
            x = jetdic[index].where(jetdic[index].time.dt.season==season).mean(dim='time')
            for i in ref.lat.values:
                for j in ref.lon.values:
                    MAE.append(float(np.abs(ref.sel(lat=i).sel(lon=j) - x.sel(lat=i).sel(lon=j))))
        results.append([reference,index,np.mean(MAE)])

In [ ]:
df = pd.DataFrame(models,columns = ['models'])
start = 0
end = len(models)
for index in models:
    df[index] = np.array(results[start:end])[:,-1].astype(float)
    start = start + len(models)
    end = end + len(models)

In [ ]:
df = df.set_index('models')

df.to_csv('Jet_2D_Linkage.csv')